In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Technical indicator library
import talib as ta

# Data Source
import yfinance as yf
from sklearn.preprocessing import StandardScaler

# For time stamps
from datetime import datetime
import math
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import os
from keras.models import load_model
import time

In [38]:
from pickle import load
df = yf.Ticker("AAPL").history('3mo',interval='1wk').filter(["Close"])
df.dropna(inplace=True)
model_weekly4 = load_model('weekly_dataset_model4.h5')
weeklyScaler = load(open('scaler_weekly.sav', 'rb'))
# Normalize data
scaled_data = weeklyScaler.fit_transform(df)
# training_data_len = 4
x_train = []
y_train = []
training_data_len = int(len(df)*0.9)

#Create Training Dataset
train_data = scaled_data[0:training_data_len, : ]
for i in range(4,len(train_data)):
    x_train.append(train_data[i-4:i,0])
    y_train.append(train_data[i,0])

#Convert x_train and y_train to numpy array
x_train , y_train = np.array(x_train), np.array(y_train)
# Reshape data
x_train  = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))

model_weekly4.fit(x_train, y_train, epochs = 10, batch_size = 1,verbose=False)
model_weekly4.save('weekly_dataset_model4.h5')
model_weekly4 = load_model('weekly_dataset_model4.h5')

# Create the testing dataset
test_data = scaled_data[training_data_len-4:, :]

# Create data sets x_test and y_test
x_test = []
y_test = scaled_data[training_data_len:, :]

for j in range(4, len(test_data)):
    x_test.append(test_data[j-4:j, 0])
new_x_test = x_test[-1]
new_x_test = np.delete(x_test[-1], 0)
new_x_test = np.append(new_x_test, y_test[-1])

# reshape data
new_x_test = np.reshape(new_x_test, (1, 4, 1))
# print(new_x_test)
# get the models predicted price values
predictions = model_weekly4.predict(new_x_test)
predictions = weeklyScaler.inverse_transform(predictions)
print(predictions[0][0])

1/1 [==============================] - 0s 111ms/step
140.11388
